In [1]:
import pandas as pd
import tensorflow as tf
import os
from PIL import Image
import io
import shutil
import random

2023-11-17 15:27:56.775754: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-17 15:27:59.168886: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-17 15:28:06.566474: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
# Training and Testing paths:
train_dir=['../data/train_labels.csv','../data/train_new.tfrecord','../data/train']
test_dir=['../data/test_labels.csv','../data/test_new.tfrecord','../data/test']


In [2]:
def class_text_to_int(row_label):
    # Map your class text to an integer. 
    if row_label == 'kalb':
        return 1
    elif row_label == 'person':
        return 2
    else:
        None

In [3]:
def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename.iloc[0])), 'rb') as fid:
        encoded_jpg = fid.read()

    filename = group.filename.iloc[0].encode('utf8')
    image_format = b'jpg'
    width = group.width.iloc[0]
    height = group.height.iloc[0]

    xmins, xmaxs, ymins, ymaxs, classes_text, classes = [], [], [], [], [], []

    for index, row in group.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        'image/filename': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename])),
        'image/source_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename])),
        'image/encoded': tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded_jpg])),
        'image/format': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_format])),
        'image/object/bbox/xmin': tf.train.Feature(float_list=tf.train.FloatList(value=xmins)),
        'image/object/bbox/xmax': tf.train.Feature(float_list=tf.train.FloatList(value=xmaxs)),
        'image/object/bbox/ymin': tf.train.Feature(float_list=tf.train.FloatList(value=ymins)),
        'image/object/bbox/ymax': tf.train.Feature(float_list=tf.train.FloatList(value=ymaxs)),
        'image/object/class/text': tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        'image/object/class/label': tf.train.Feature(int64_list=tf.train.Int64List(value=classes)),

    }))
    return tf_example

In [4]:
def main(csv_input, output_path, image_dir):
    writer = tf.io.TFRecordWriter(output_path)
    examples = pd.read_csv(csv_input)
    grouped = examples.groupby('filename')
    for filename, group in grouped:
        tf_example = create_tf_example(group, image_dir)
        writer.write(tf_example.SerializeToString())
    writer.close()

In [9]:
main(test_dir[0], test_dir[1], test_dir[2])

In [10]:
main(train_dir[0], train_dir[1], train_dir[2])

In [11]:
# Counting number of records in tensorflow record file
import tensorflow as tf

record_file = train_dir[1]
count = 0

for record in tf.data.TFRecordDataset(record_file):
    count += 1

print(f"Total samples in TFRecord: {count}")


Total samples in TFRecord: 1398
